# Practical 3: Supply chain linkages

In [1]:
# import modules
from functools import reduce
from operator import add

import pandas as pd
import numpy as np

## Exercise 1
### Declare input values

#### Z: Inter-industry transactions of intermediate products (unit: €/year)

Intermediate products: goods and services as inputs in the production processes.

In [2]:
# Inter-industry transactions
sectors = ["Agriculture", "Manufacturing", "Services"]
Z_values = [
    [0.6,2.6,0.5],
    [0.8, 30.6, 7.8],
    [0.9,12.1,23]
]

Z = pd.DataFrame(Z_values, index=sectors, columns=sectors)
Z

,Agriculture,Manufacturing,Services
Agriculture,0.6,2.6,0.5
Manufacturing,0.8,30.6,7.8
Services,0.9,12.1,23.0


#### V: Value added (unit: €/year)

- Economic gains generated by producing goods and services.
- Main components: wages, taxes minus subsidies, capital profits, etc
- $v$: value added pet unit per year
- $\mathbf{V}$: value added per year

In [3]:
# Value added (last row of the first table)
V_values = [3.30, 22.4, 52.5]

V = pd.Series(V_values, index=sectors)
V

Agriculture       3.3
Manufacturing    22.4
Services         52.5
dtype: float64

#### Y: Final demand (unit: €/year)

In [4]:
# Final demand vector of products purchased by final consumers
y_values = [1.9, 28.5, 47.8]

y = pd.Series(y_values, index=sectors)
y

Agriculture       1.9
Manufacturing    28.5
Services         47.8
dtype: float64

### Calculations
#### x: Gross output (unit: €/year)

**How does a sector distributes its outputs?**  
total outputs (production) = intermediate output + final product output 

$x_i = \sum_{j=0}^n z_{ij} + y_i$

In [5]:
# Method 1
x_out = Z.sum(axis=1) + y
x_out

Agriculture       5.6
Manufacturing    67.7
Services         83.8
dtype: float64

**What does a sector consume?**  
Gross output (production recipe) of Sector j: intermediate prices + total value-added expenditures of Sector j

$x_j = \sum_{j=0}^n z_{ij} + v_j$

In [6]:
# Method 2
x_in = Z.sum() + V
x_in

Agriculture       5.6
Manufacturing    67.7
Services         83.8
dtype: float64

In [7]:
# Ensure method 1 = method 2
if not np.isclose(x_in, x_out).all():
    raise ValueError(f"discrepancies between\nx_in\n{x_in}\n\nx_out\n{x_out}")
x = x_out

#### A: Technical coefficient (unit: €/€)

$\mathbf{A} = \mathbf{Z}\hat{\mathbf{x}}^{-1}$  
$a_{i,j} = z_{i,j}/x_j$  
$z_{i,j} = a_{i,j} x_j$  

$aij$ : direct requirements of sector i per € output of sector j  

In [51]:
# A (Unit: Million euro/Million euro)
A = Z @ np.linalg.inv(np.diag(x))  # @ = np.matmul
A.rename(columns=dict(enumerate(sectors)), inplace=True)

# alternative method
A = Z.divide(x)  # same result for Z/x and Z/(x.T)

A.round(3)

,Agriculture,Manufacturing,Services
Agriculture,0.107,0.038,0.006
Manufacturing,0.143,0.452,0.093
Services,0.161,0.179,0.274


#### L: Leontief inverse
##### by inverse

$\mathbf{Z} + \mathbf{y} = \mathbf{x}$  
$\mathbf{Ax} + \mathbf{y} = \mathbf{x}$  
$\mathbf{y} = (\mathbf{I}-\mathbf{A})\mathbf{x}$  
$(\mathbf{I}-\mathbf{A})^{-1}(\mathbf{I}-\mathbf{A})\mathbf{x}=(\mathbf{I}-\mathbf{A})^{-1}\mathbf{y}$  
$\mathbf{x} = (\mathbf{I}-\mathbf{A})^{-1}\mathbf{y} = \mathbf{L}\mathbf{y}$

$lij$ : total requirements of sector i per € final demand of sector j

In [17]:
# Create an identity matrix the same order
Id = np.identity(len(sectors))

# Compute leontief inverse matrix in the Demand-pull model
L_values = np.linalg.inv((Id - A))
L = pd.DataFrame(L_values, index=sectors, columns=sectors)

# visualize
L.round(3)

,Agriculture,Manufacturing,Services
Agriculture,1.137,0.086,0.020
Manufacturing,0.354,1.931,0.251
Services,0.339,0.495,1.445


##### by series expansion

L covers supply chains of inifinte lengths. Power expansions stops at a certain depth

$$(\mathbf{I}-\mathbf{A})^{-1} = \mathbf{I} + \mathbf{A} + \mathbf{A}^2 + \mathbf{A}^3 + ... + \mathbf{A}^n + ... = \lim_{n \to +\infty} \sum_{k=0}^{n} \mathbf{A}^k$$

The power series converges only under certain conditions:
- $\mathbf{A}$ ≥ 0 (coefficients matrix contains only non-negative terms)
- The system produces more output than it requires inputs $N(\mathbf{A}) < 1$
- $|\mathbf{I} − \mathbf{A}| > 0$
- More generally, the [Hawkins-Simon conditions](https://www.wikiwand.com/en/Hawkins%E2%80%93Simon_condition)

the limit sum convergs if all of the eigenvalues of A have absolute value smaller than 1 (?)

In [23]:
def leontief_power_series(A_mat, depth=3):
    # Function to calculate L through series expansion.
    Id_mat = pd.DataFrame(np.identity(len(A_mat)), index=A.index, columns=A.columns)
    
    res = [None]*(depth+1)
    res[0] = Id_mat

    for i in range(1, depth+1):
        res[i] = res[i-1] @ A_mat
        # Comment below to hide inbetween steps
        print(f"{'A'*i}\n{res[i]}\n\n")

    return reduce(add, res)

In [24]:
leontief_power_series(A, depth=3)

A
               Agriculture  Manufacturing  Services
Agriculture       0.107143       0.038405  0.005967
Manufacturing     0.142857       0.451994  0.093079
Services          0.160714       0.178730  0.274463


AA
               Agriculture  Manufacturing  Services
Agriculture       0.017925       0.022540  0.005852
Manufacturing     0.094836       0.226421  0.068470
Services          0.086862       0.136012  0.092925


AAA
               Agriculture  Manufacturing  Services
Agriculture       0.006081       0.011922  0.003811
Manufacturing     0.053511       0.118221  0.040433
Services          0.043671       0.081421  0.038682




,Agriculture,Manufacturing,Services
Agriculture,1.131149,0.072867,0.015629
Manufacturing,0.291204,1.796636,0.201982
Services,0.291248,0.396162,1.406070


#### Demand pull

Price is fixed and the quantities change:

- Given $\mathbf{y}$, we compute $\mathbf{x} = \mathbf{L}\mathbf{y}$  
- Or given $\Delta\mathbf{y}$, we compute $\Delta\mathbf{x} = \mathbf{L}\Delta\mathbf{y}$  

In [ ]:
# Check that the total product output is equal to the one we have already calculated
np.isclose(x, L @ y).all()

#### 'Cost-push' model (or 'Price' model)

Quantities are fixed, and the prices change:
- Given $\mathbf{v'} (= v_j'/x_j)$, we compute $\mathbf{p} = \mathbf{L}'\mathbf{v'}$  
- Or given $\Delta \mathbf{v} (= \Delta v_j/x_j)$, we compute $\Delta\mathbf{p} = \mathbf{L}'\Delta \mathbf{v}$  

with:
- $'$ the transpose
- $\mathbf{p}_{i}$ the price per unit of product i
- $(1- \mathbf{A}')^{-1} = \mathbf{L}'$ 
-  $\Delta\mathbf{v}$ the changes in taxes, wages, profits, capital payment.  


Intermediary steps:  
$\mathbf{p} = \mathbf{A}'\mathbf{p} + \mathbf{v}'$  
$\mathbf{p} - \mathbf{A}'\mathbf{p} = \mathbf{v}'$  
$(1- \mathbf{A}')\mathbf{p} = \mathbf{v}'$  
$\mathbf{p} = (1- \mathbf{A}')^{-1}\mathbf{v}'$  
$\mathbf{p} = \mathbf{L}'\mathbf{v}'$  

We can therefore calculate for relative price changes:  

$\mathbf{p}^{*} = \mathbf{L}'\mathbf{v}^{*'}$  
or  
$\Delta\mathbf{p} = \mathbf{L}'(\mathbf{v}'-\mathbf{v}^{*'}) = \mathbf{L}'\Delta\mathbf{v}'$  

In [25]:
# Calculate the leontief inverse matrix in the price model
L_prime_values = np.linalg.inv((Id - A.T))
L_prime = pd.DataFrame(L_prime_values, index=sectors, columns=sectors)
L_prime

,Agriculture,Manufacturing,Services
Agriculture,1.137500,0.354144,0.339209
Manufacturing,0.086382,1.931377,0.494912
Services,0.020436,0.250688,1.444571


In [26]:
# calculate the value added coefficients by dividing the value added by total product output
v_prime = V.T/x

# and Check that p = 1 (since v* = v)
p = L_prime @ v_prime
p

Agriculture      1.0
Manufacturing    1.0
Services         1.0
dtype: float64

#### G: Ghosh quantity model

$\mathbf{B} = \hat{\mathbf{x}}^{-1} \mathbf{Z}$  
$\mathbf{G} = (\mathbf{I}-\mathbf{B})^{-1}$  

with:
- $\mathbf{B}$, the allocation coefficients matrix (or direct-output coefficients)
- $b_{i,*}$ the distribution of sector i’s outputs across sectors that purchase inputs from i
- $\mathbf{G}$, the Ghosh inverse

In [56]:
# B (Unit: Million euro/Million euro)
B = np.linalg.inv(np.diag(x)) @ Z 
B.rename(index=dict(enumerate(sectors)), inplace=True)

# alternative
# B = Z.divide(x.values[:, None])  # (Z/(x.T) = Z/x = A)

B.round(3)

,Agriculture,Manufacturing,Services
Agriculture,0.107,0.464,0.089
Manufacturing,0.012,0.452,0.115
Services,0.011,0.144,0.274


In [65]:
# Calculate the Ghosh inverse matrix
G_values = np.linalg.inv((Id - B))
G = pd.DataFrame(G_values, index=sectors, columns=sectors)
G.round(3)

,Agriculture,Manufacturing,Services
Agriculture,1.137,1.044,0.306
Manufacturing,0.029,1.931,0.310
Services,0.023,0.400,1.445


In [66]:
# Check that the total product input is equal to the one we have already calculated x=G'v
x_g = G.T @ V

if not np.isclose(x, x_g).all():
    raise ValueError(f"discrepancies between\nx\n{x}\n\nx_g\n{x_g}")

## Exercise 2
### Backward linkages

In [68]:
# Direct backward linkages
A.sum()

Agriculture      0.410714
Manufacturing    0.669129
Services         0.373508
dtype: float64

In [70]:
# Normalized Direct backward linkages
norm_dir_back_link = A.sum() / (A.sum().sum() / len(sectors))
norm_dir_back_link

Agriculture      0.847794
Manufacturing    1.381212
Services         0.770994
dtype: float64

In [69]:
# Total backward linkages
L.sum()

Agriculture      1.830853
Manufacturing    2.512672
Services         1.715695
dtype: float64

In [71]:
# Normalized Total backward linkages
norm_tot_back_link = L.sum() / (L.sum().sum() / len(sectors))
norm_tot_back_link

Agriculture      0.906479
Manufacturing    1.244057
Services         0.849463
dtype: float64

### Forward linkages

In [73]:
# Direct forward linkages
B.sum(axis=1)

Agriculture      0.660714
Manufacturing    0.579025
Services         0.429594
dtype: float64

In [74]:
# Normalized Direct forward linkages
norm_dir_fwd_link = B.sum(axis=1) / (B.sum(axis=1).sum() / len(sectors))
norm_dir_fwd_link

Agriculture      1.187386
Manufacturing    1.040580
Services         0.772034
dtype: float64

In [75]:
# Total forward linkages
G.sum(axis=1)

Agriculture      2.487614
Manufacturing    2.270976
Services         1.867067
dtype: float64

In [76]:
# Normalized Total forward linkages
norm_tot_fwd_link = G.sum(axis=1) / (G.sum(axis=1).sum() / len(sectors))
norm_tot_fwd_link

Agriculture      1.126355
Manufacturing    1.028265
Services         0.845380
dtype: float64

### Sector interdependencies

![Linkage interdependencies](../img/linkages.png)

In [86]:
direct_interdependencies = pd.concat([
        norm_dir_back_link,
        norm_dir_fwd_link],
    axis=1)

direct_interdependencies.columns = [
        "direct back link",
        "direct forward link",
    ]
direct_interdependencies.round(3)

,direct back link,direct forward link
Agriculture,0.848,1.187
Manufacturing,1.381,1.041
Services,0.771,0.772


In [87]:
total_interdependencies = pd.concat([
        norm_tot_back_link,
        norm_tot_fwd_link],
    axis=1)

total_interdependencies.columns = [
        "total back link",
        "total forward link"
    ]
total_interdependencies.round(3)

,total back link,total forward link
Agriculture,0.906,1.126
Manufacturing,1.244,1.028
Services,0.849,0.845
